In [48]:
import os
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import keras.preprocessing.image as Image
from keras.models import Model
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
model = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=None)

In [31]:
#def save_bottleneck_features():
    #"""VGG16にDog vs Catの訓練画像、バリデーション画像を入力し、
    #ボトルネック特徴量（FC層の直前の出力）をファイルに保存する"""

# VGG16モデルと学習済み重みをロード
# Fully-connected層（FC）はいらないのでinclude_top=False）
base_model = VGG16(include_top=False, 
                   weights='imagenet')
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [55]:
model = Model(
    inputs=base_model.input,
    outputs=base_model.get_layer("block4_pool").output
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [43]:
img = image.load_img("/home/baba/test_VGG16/gnct_symbol.jpg",
                    target_size = (224,224))
x = image.img_to_array(img)
x = np.expand_dims(x,axis = 0)
x = x/255

In [8]:
def converter(xarray, yarray, num, outputnum):
    #numで指定されたラベルの値の該当するデータを抽出し，成形する
    xarray = xarray[np.where(yarray == num)[0]]
    xarray = xarray.astype('float32') / 255.0
    xarray = np.reshape(xarray, (len(xarray),outputnum))
    return xarray

In [20]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train_1 = converter(x_train, y_train, 1, 784)
x_test_1 = converter(x_test, y_test, 1, 784)
x_test_7 = converter(x_test, y_test, 7, 784)
x_train_1.shape[0]

6742

In [23]:
x_train_1 = x_train_1.reshape(x_train_1.shape[0],28,28,1)

In [54]:
bottleneck_features_train = model.predict(x)
bottleneck_features_train = bottleneck_features_train * 255.0

array([6.98144760e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.07047005e+01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 9.74921112e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 8.75208511e+01, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 5.37456551e+01, 0.00000000e+00,
       0.00000000e+00, 8.80328979e+01, 5.28911705e+01, 0.00000000e+00,
       0.00000000e+00, 7.02702560e+01, 0.00000000e+00, 2.57650185e+01,
       1.70228882e+01, 0.00000000e+00, 0.00000000e+00, 1.16031784e+02,
      

In [39]:
np.save(os.path.join("/home/baba/test_VGG16/", 'bottleneck_features_validation.npy'),
            bottleneck_features_train)

In [ ]:
    # ジェネレータの設定
    datagen = ImageDataGenerator(rescale=1.0 / 255)

    # Dog vs Catのトレーニングセットを生成するジェネレータを作成
    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

    # ジェネレータから生成される画像を入力し、VGG16の出力をファイルに保存
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples)
    np.save(os.path.join(result_dir, 'bottleneck_features_train.npy'),
            bottleneck_features_train)

    # Dog vs Catのバリデーションセットを生成するジェネレータを作成
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode=None,
        shuffle=False)

    # ジェネレータから生成される画像を入力し、VGG16の出力をファイルに保存
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples)
    np.save(os.path.join(result_dir, 'bottleneck_features_validation.npy'),
            bottleneck_features_validation)

In [ ]:
def train_top_model():
    """VGGのボトルネック特徴量を入力とし、Dog vs Catの正解を出力とするFCネットワークを訓練"""
    # 訓練データをロード
    # ジェネレータではshuffle=Falseなので最初の1000枚がcats、次の1000枚がdogs
    train_data = np.load(os.path.join(result_dir, 'bottleneck_features_train.npy'))
    train_labels = np.array([0] * int(nb_train_samples / 2) + [1] * int(nb_train_samples / 2))

    # (2000, 4, 4, 512)
    print(train_data.shape)

    # バリデーションデータをロード
    validation_data = np.load(os.path.join(result_dir, 'bottleneck_features_validation.npy'))
    validation_labels = np.array([0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples / 2))

    # (800, 4, 4, 512)
    print(validation_data.shape)

    # FCネットワークを構築
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                        nb_epoch=nb_epoch,
                        batch_size=32,
                        validation_data=(validation_data, validation_labels))

    model.save_weights(os.path.join(result_dir, 'bottleneck_fc_model.h5'))
    save_history(history, os.path.join(result_dir, 'history_extractor.txt'))

In [ ]:
if __name__ == '__main__':
    # 訓練データとバリデーションデータのボトルネック特徴量の抽出
    save_bottleneck_features()

    # ボトルネット特徴量でFCネットワークを学習